In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
DATA_PATH = 'processed_data.csv'

In [4]:
data_df = pd.read_csv(DATA_PATH)

In [5]:
data_df

,Unnamed: 0,current_province,current_city,current_county,current_members_live_with,gender,year_born,edu_level,hometown_code,hometown,...,average_family_cost_per_month,average_family_income_per_month,year_first_flow,month_first_flow,first_flow_code,first_flow_location,num_flows_total,if_change_household_local,if_stay,how_long_to_stay
0,1,广东省,深圳市,光明新区,4,2,1989,4,360681,江西省鹰潭市贵溪市,...,10000.0,NaN,2008,11,360111,江西省南昌市青山湖区,4,1,1,3.0
1,2,广东省,深圳市,光明新区,3,2,1981,5,360402,江西省九江市濂溪区,...,60000.0,NaN,2000,5,440305,广东省深圳市南山区,3,1,1,5.0
2,3,福建省,厦门市,同安区,4,2,1986,4,511623,四川省广安市邻水县,...,40000.0,NaN,2001,3,350503,福建省泉州市丰泽区,2,1,1,5.0
3,4,新疆生产建设兵团,第六师,军户农场,3,1,1973,3,652301,新疆维吾尔自治区昌吉回族自治州昌吉市,...,5000.0,NaN,2000,3,660613,新疆兵团第六师军户农场,1,2,1,5.0
4,5,北京市,北京市,朝阳区,2,2,1984,5,511702,四川省达州市通川区,...,9000.0,NaN,2003,9,510107,四川省成都市武侯区,2,1,1,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169984,169985,上海市,上海市,闵行区,3,2,1959,2,330602,浙江省绍兴市越城区,...,10000.0,100000.0,1982,2,310112,上海市市辖区闵行区,1,2,1,5.0
169985,169986,安徽省,宣城市,宁国市,4,1,1976,3,341823,安徽省宣城市泾县,...,15000.0,120000.0,1996,2,310112,上海市市辖区闵行区,3,2,1,5.0
169986,169987,北京市,北京市,海淀区,4,2,1977,6,131022,河北省廊坊市固安县,...,120000.0,180000.0,1997,7,110105,北京市市辖区朝阳区,1,1,1,5.0
169987,169988,浙江省,杭州市,江干区,4,2,1971,3,330213,浙江省宁波市奉化区,...,100000.0,200000.0,1992,3,330104,浙江省杭州市江干区,2,1,1,5.0


In [6]:
data_df.size

3739758

In [10]:
data_df["first_flow_location"].nunique()

3087